In [1]:
!nvidia-smi
!nvcc --version

Thu Oct  6 10:52:28 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 512.15       Driver Version: 512.15       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:04:00.0  On |                  N/A |
|  0%   57C    P8    18W / 220W |   1016MiB /  8192MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!git clone https://github.com/sberbank-ai/sber-swap.git
%cd sber-swap
!wget -P ./arcface_model https://github.com/sberbank-ai/sber-swap/releases/download/arcface/backbone.pth
!wget -P ./arcface_model https://github.com/sberbank-ai/sber-swap/releases/download/arcface/iresnet.py
!wget -P ./insightface_func/models/antelope https://github.com/sberbank-ai/sber-swap/releases/download/antelope/glintr100.onnx
!wget -P ./insightface_func/models/antelope https://github.com/sberbank-ai/sber-swap/releases/download/antelope/scrfd_10g_bnkps.onnx
!wget -P ./weights https://github.com/sberbank-ai/sber-swap/releases/download/sber-swap-v2.0/G_unet_2blocks.pth
!wget -P ./weights https://github.com/sberbank-ai/sber-swap/releases/download/super-res/10_net_G.pth

In [12]:
!pip install mxnet-cu101mkl
!pip install onnxruntime-gpu==1.8
!pip install insightface==0.2.1
!pip install kornia==0.5.4

  Using cached numpy-1.16.6-cp39-cp39-win_amd64.whl
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.3
    Uninstalling numpy-1.23.3:
      Successfully uninstalled numpy-1.23.3


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tifffile 2022.8.12 requires numpy>=1.19.2, but you have numpy 1.16.6 which is incompatible.
tensorflow-gpu 2.8.0 requires keras<2.9,>=2.8.0rc0, but you have keras 2.7.0 which is incompatible.
tensorflow-gpu 2.8.0 requires numpy>=1.20, but you have numpy 1.16.6 which is incompatible.
tensorboard 2.8.0 requires requests<3,>=2.21.0, but you have requests 2.18.4 which is incompatible.
scipy 1.8.0 requires numpy<1.25.0,>=1.17.3, but you have numpy 1.16.6 which is incompatible.
scikit-image 0.19.3 requires numpy>=1.17.0, but you have numpy 1.16.6 which is incompatible.
quimb 1.4.0 requires numpy>=1.17, but you have numpy 1.16.6 which is incompatible.
qiskit-terra 0.21.2 requires numpy>=1.17, but you have numpy 1.16.6 which is incompatible.
qiskit-ibmq-provider 0.19.2 requires requests>=2.19, but you have requests 2.18.4

  Using cached numpy-1.23.3-cp39-cp39-win_amd64.whl (14.7 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.16.6
    Uninstalling numpy-1.16.6:
      Successfully uninstalled numpy-1.16.6


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-gpu 2.8.0 requires keras<2.9,>=2.8.0rc0, but you have keras 2.7.0 which is incompatible.
tensorboard 2.8.0 requires requests<3,>=2.21.0, but you have requests 2.18.4 which is incompatible.
qiskit-ibmq-provider 0.19.2 requires requests>=2.19, but you have requests 2.18.4 which is incompatible.
numba 0.56.2 requires setuptools<60, but you have setuptools 65.4.1 which is incompatible.
mxnet-cu101mkl 1.5.0 requires numpy<1.17.0,>=1.8.2, but you have numpy 1.23.3 which is incompatible.


In [16]:
import cv2
import torch
import time
import os
import matplotlib.pyplot as plt
import sys

sys.path.append('./utils')

from IPython.display import HTML
from base64 import b64encode

from utils.inference.image_processing import crop_face, get_final_image, show_images
from utils.inference.video_processing import read_video, get_target, get_final_video, add_audio_from_another_video, face_enhancement
from utils.inference.core import model_inference

from network.AEI_Net import AEI_Net
from coordinate_reg.image_infer import Handler
from insightface_func.i import Face_detect_crop
from arcface_model.iresnet import iresnet100
from models.pix2pix_model import Pix2PixModel
from models.config_sr import TestOptions
from .autonotebook import tqdm as notebook_tqdm

ModuleNotFoundError: No module named 'utils'

In [ ]:
app = Face_detect_crop(name='antelope', root='./insightface_func/models')
app.prepare(ctx_id= 0, det_thresh=0.6, det_size=(640,640))

G = AEI_Net(backbone='unet', num_blocks=2, c_id=512)
G.eval()
G.load_state_dict(torch.load('weights/G_unet_2blocks.pth', map_location=torch.device('cpu')))
G = G.cuda()
G = G.half()

netArc = iresnet100(fp16=False)
netArc.load_state_dict(torch.load('arcface_model/backbone.pth'))
netArc=netArc.cuda()
netArc.eval()

handler = Handler('./coordinate_reg/model/2d106det', 0, ctx_id=0, det_size=640)

use_sr = True
if use_sr:
    os.environ['CUDA_VISIBLE_DEVICES'] = '0'
    torch.backends.cudnn.benchmark = True
    opt = TestOptions()
    model = Pix2PixModel(opt)
    model.netG.train()

In [ ]:
target_type = 'video'

source_path = 'examples/images/jin.png'
path_to_video = 'examples/videos/dj.mp4'

source_full = cv2.imread(source_path)
OUT_VIDEO_NAME = "examples/results/result.mp4"
crop_size = 224

try:
    source = crop_face(source_full, app, crop_size)[0]
    source = [source[:, :, ::-1]]
    print("Everything is ok!")
except TypeError:
    print("Bad source images")

if target_type == 'image':
    target_full = cv2.imread(target_path)
    full_frames = [target_full]
else:
    full_frames, fps = read_video(path_to_video)
target = get_target(full_frames, app, crop_size)

In [ ]:
batch_size = 40

START_TIME = time.time()

final_frames_list, crop_frames_list, full_frames, tfm_array_list = model_inference(
    full_frames,
    source,
    target,
    netArc,
    G,
    app,
    set_target = False,
    crop_size=crop_size,
    BS=batch_size
)

if use_sr:
  final_frames_list = face_enhancement(final_frames_list, model)

if target_type == 'video':
  get_final_video(
      final_frames_list,
      crop_frames_list,
      full_frames,
      tfm_array_list,
      OUT_VIDEO_NAME,
      fps,
      handler
  )

  add_audio_from_another_video(path_to_video, OUT_VIDEO_NAME, "audio")

  print(f'Full pipeline took {time.time() - START_TIME}')
  print(f"Video saved with path {OUT_VIDEO_NAME}")
else:
  result = get_final_image(
      final_frames_list,
      crop_frames_list,
      full_frames[0],
      tfm_array_list,
      handler
  )
  cv2.imwrite('examples/results/result.png', result)

In [ ]:
if target_type == 'image':
  show_images(
      [
          source[0][:,:,::-1],
          target_full,
          result
      ],
      [
          'Source Image',
          'Target Image',
          'Swapped Image'
      ],
      figsize=(20, 15)
  )
else:
  video_file = open(OUT_VIDEO_NAME, "r+b").read()
  video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
  
  HTML(f"""<video width={800} controls><source src="{video_url}"></video>""")